In [2]:
import math
import torch
import torch.nn as nn
import torch.nn.init as init

### 建立block

In [3]:
class Block(nn.Module):
    def __init__(self, in_ch,out_ch):
        super(Block, self).__init__()
        self.conv1 = nn.Conv2d(in_ch, out_ch, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(out_ch)
        self.relu1 = nn.ReLU(inplace=True)
    def forward(self, x):
        out = self.relu1(self.bn1(self.conv1(x)))
        return out

### 建立layer

In [ ]:
def make_layers(in_channels, layer_list):
    """
    in_channels: 3the number of input's channels
    layer_list: as : [64,64]
    """
    layers = []
    for v in layer_list:
        layers += [Block(in_channels, v)]
        in_channels = v
    return nn.Sequential(*layers)


class Layer(nn.Module):
    def __init__(self, in_channels, layer_list):
        super(Layer, self).__init__()
        self.layer = make_layers(in_channels, layer_list)
        
    def forward(self, x):
        out = self.layer(x)
        return out

### 建立VGG-19BN--encode模型

* 'A': [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
* 'B': [64, 64, 'M', 128, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
* 'D': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M'],
* 'E': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M', 512, 512, 512, 512, 'M'],

In [ ]:
class VGG19BN(nn.Module):
    '''
    VGG model 
    '''
    def __init__(self):
        super(VGG19BN, self).__init__()
        # 224
        self.layer1 = Layer(3, [64, 64])
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        #112
        self.layer2 = Layer(64, [128, 128])
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        #56
        self.layer3 = Layer(128, [256, 256, 256, 256])
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)
        #28
        self.layer4 = Layer(256, [512, 512, 512, 512])
        self.pool4 = nn.MaxPool2d(kernel_size=2, stride=2)
        #14
        self.layer5 = Layer(512, [512, 512, 512, 512])
        self.pool5 = nn.MaxPool2d(kernel_size=2, stride=2)
        #7
        # N,C,H,W,
        # N,C*H*W
#         self.flat = nn.Flatten()
#         self.fc1 = nn.Linear(7*7*512, 4096)
#         self.fc2 = nn.Linear(4096, 4096)
#         self.classifer = nn.Linear(4096, 1000)
    def forward(self, x):
        f1 = self.pool1(self.layer1(x))
        f2 = self.pool2(self.layer2(f1))
        f3 = self.pool3(self.layer3(f2))
        f4 = self.pool4(self.layer4(f3))
        f5 = self.pool5(self.layer5(f4))
        # p = self.classifer(self.fc2(self.fc1(self.flat(f5))))
        
        return f5

In [ ]:
x = torch.randn((10,3, 224,224))
model = VGG19BN()


In [ ]:
model.eval()
y = model(x)
y.size()